In [1]:
import os

In [2]:
%pwd

'd:\\Projects\\Kidney_Disesae_Classification_with-MLFLOW_-_DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Projects\\Kidney_Disesae_Classification_with-MLFLOW_-_DVC'

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/saisuryachandraprasad/Kidney_Disesae_Classification_with-MLFLOW_-_DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "saisuryachandraprasad"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "2dcd3ae6674d1754dae248d34dd8cce9a5c5aa0c"

In [6]:
from dataclasses import dataclass 
from pathlib import Path

In [7]:
@dataclass(frozen= True)
class ModelEvaluationConfig:

    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [8]:
from src.Kidney_Disease_Classification.constants import *
from src.Kidney_Disease_Classification.utils.common import read_yaml, create_directory, save_json

In [9]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        evaluation_config = ModelEvaluationConfig(
            path_of_model = "artifacts/training/model.h5",
            training_data = "artifacts/data_ingestion/kidney-ct-scan-image",
            all_params = self.params,
            mlflow_uri = "https://dagshub.com/saisuryachandraprasad/Kidney_Disesae_Classification_with-MLFLOW_-_DVC.mlflow",
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )

        return evaluation_config

In [10]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [11]:
class Evaluation:
    def __init__(self,config : ModelEvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )


        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset= "validation",
            shuffle= False,
            **dataflow_kwargs
            )

    @staticmethod
    def load_model(path : Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    
    def save_score(self):
        scores = {"loss":self.score[0], "accuracy":self.score[1]}
        save_json(path= Path("scores.json"), data=scores)

    def log_into_mlfow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                "loss": self.score[0], "accuracy":self.score[1]
            })

            if tracking_url_type_store != "file":

                mlflow.keras.log_model(self.model, "model", registered_model_name = "VGG16Model")

            else:
                mlflow.keras.log_model(self.model, "model")

In [12]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    evaluation = Evaluation(config = model_evaluation_config)
    evaluation.evaluation()
    evaluation.save_score()
    evaluation.log_into_mlfow()

except Exception as e:
    raise e

[2023-11-16 13:33:17,216  - INFO - common - Reading done from config\config.yaml file]
[2023-11-16 13:33:17,220  - INFO - common - Reading done from params.yaml file]
[2023-11-16 13:33:17,222  - INFO - common - Directory is created for artifacts]
Found 93 images belonging to 2 classes.
6/6 [==============================] - 11s 2s/step - loss: 27.7679 - accuracy: 0.5484
[2023-11-16 13:33:29,420  - INFO - common - data is saved in json format]


2023/11/16 13:33:33 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2023-11-16 13:33:35,196  - WARNING - save - Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
[2023-11-16 13:33:35,934  - INFO - builder_impl - Assets written to: C:\Users\SAICHA~1\AppData\Local\Temp\tmpomz9e32v\model\data\model\assets]


d:\Projects\Kidney_Disesae_Classification_with-MLFLOW_-_DVC\venv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2023/11/16 13:34:46 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.
